In [2]:
import duckdb
from datetime import datetime, timedelta
import pandas as pd

# Get yesterday's date
yesterday = datetime.today() - timedelta(days=1)

# Manually format the db-name
yesterday_db = f"data/{yesterday.month}_{yesterday.day}_{yesterday.year}_A.db"

# Connect to the DuckDB database
con = duckdb.connect('yesterday_db')

# Define the updated SQL query
query = '''
WITH numbered AS (
    SELECT number,
           COUNT(*) AS count,
           ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS seq
    FROM (
        SELECT num1 AS number FROM numbers_data
        UNION ALL
        SELECT num2 FROM numbers_data
        UNION ALL
        SELECT num3 FROM numbers_data
        UNION ALL
        SELECT num4 FROM numbers_data
        UNION ALL
        SELECT num5 FROM numbers_data
    ) combined_numbers
    GROUP BY number
),
top_number AS (
    SELECT number
    FROM numbered
    WHERE seq BETWEEN 1 AND 5
    ORDER BY RANDOM()
    LIMIT 1
),
range_6_35 AS (
    SELECT number
    FROM numbered
    WHERE seq BETWEEN 6 AND 35
    ORDER BY RANDOM()
    LIMIT 3
),
range_36_45 AS (
    SELECT number
    FROM numbered
    WHERE seq BETWEEN 36 AND 45
    ORDER BY RANDOM()
    LIMIT 1
),
pick_5 AS (
    SELECT number FROM top_number 
    UNION ALL
    SELECT number FROM range_6_35 
    UNION ALL
    SELECT number FROM range_36_45
)

FROM pick_5
ORDER BY number;
'''

# Execute the query and fetch results
result = con.execute(query).fetchall()

# Convert result to a space-separated string
formatted_result = ' '.join(str(row[0]) for row in result)

# Print the formatted result
print(formatted_result)

# Close the DuckDB connection
con.close()


4 8 17 32 45
